In [114]:
import Pkg
Pkg.add("TimeSeries")

    Updating registry at `C:\Users\matsz\.julia\registries\General.toml`
   Resolving package versions...
   Installed MutableArithmetics ──────── v1.4.4
   Installed Libmount_jll ────────────── v2.40.1+0
   Installed MvNormalCDF ─────────────── v0.3.2
   Installed JpegTurbo_jll ───────────── v3.0.3+0
   Installed GR_jll ──────────────────── v0.73.5+0
   Installed OffsetArrays ────────────── v1.14.0
   Installed Unitful ─────────────────── v1.19.1
   Installed OpenSSL ─────────────────── v1.4.3
   Installed Polyester ───────────────── v0.7.14
   Installed Contour ─────────────────── v0.6.3
   Installed RecursiveArrayTools ─────── v3.18.1
   Installed BSplineKit ──────────────── v0.17.2
   Installed HTTP ────────────────────── v1.10.8
   Installed Cairo_jll ───────────────── v1.18.0+2
   Installed ColorSchemes ────────────── v3.25.0
   Installed Fontconfig_jll ──────────── v2.13.96+0
   Installed Libgpg_error_jll ────────── v1.49.0+0
   Installed Xorg_libSM_jll ──────────── v1.2.4+0
   

In [175]:
using DataFrames, FredData, XLSX, SHA, Dates, CSV, TimeSeries, StatsBase, HiddenMarkovModels, Distributions

In [305]:
function regime_summary(loglikehood,hmm_est, freq = 1)
    return DataFrame(Dict(
      "mu1" => mean(obs_distributions(hmm_est)[1]) * freq,
      "sig1" => std(obs_distributions(hmm_est)[1])*  freq^0.5,
      "mu2" => mean(obs_distributions(hmm_est)[2]) * freq,
      "sig2" => std(obs_distributions(hmm_est)[2])* freq^0.5,
      "p11" => transition_matrix(hmm_est)[1,1],
      "p21" => transition_matrix(hmm_est)[2,1],
      "loglikehood" => last(loglikehood)
  ))
  end

regime_summary (generic function with 2 methods)

In [307]:
usdpln = CSV.read("usdpln_m.csv", DataFrame)[:,[:Data,:Zamkniecie]]
usdpln = rename(usdpln,[:date, :usdpln])
usdpln.date = Date.(usdpln.date,dateformat"u dd, yyyy") 
usdpln

Row,date,usdpln
,Date,Float64
1,2000-12-29,4.128
2,2001-01-31,4.08
3,2001-02-28,4.0375
4,2001-03-30,4.0903
5,2001-04-30,3.9618
6,2001-05-31,3.9824
7,2001-06-29,4.0069
8,2001-07-31,4.2558
9,2001-08-31,4.239


In [308]:
acwi = DataFrame(XLSX.readdata("acwi_15.05.24.xlsx","net!A8:B288"), [:date,:acwi])
acwi.date = Date.(acwi.date,dateformat"u dd, yyyy") 
acwi.acwi = parse.(Float64,string.(acwi.acwi))
acwi

Row,date,acwi
,Date,Float64
1,2000-12-29,100.0
2,2001-01-31,102.514
3,2001-02-28,93.869
4,2001-03-30,87.516
5,2001-04-30,93.853
6,2001-05-31,92.75
7,2001-06-29,89.882
8,2001-07-31,88.447
9,2001-08-31,84.352


In [309]:
data = leftjoin(acwi,usdpln, on= :date)
data.acwi_pln = data.acwi .* data.usdpln


281-element Vector{Float64}:
  412.8
  418.25712
  378.9960875
  357.9666948
  371.8268154
  369.36760000000004
  360.1481858
  376.4127426
  357.568128
  323.9934875
    ⋮
 1503.37613752
 1526.71381512
 1428.2824351999998
 1480.1930220000002
 1526.90645538
 1562.9809198500002
 1625.9419341
 1675.18580625
 1649.7202854999998

In [310]:
returns = log.(data.acwi_pln[2:end]./data.acwi_pln[1:end-1])
returns = DataFrame([data.date[2:end],returns],[:date,:acwipln_returns,])


Row,date,acwipln_returns
,Date,Float64
1,2001-01-31,0.0131331
2,2001-02-28,-0.0985705
3,2001-03-30,-0.0570859
4,2001-04-30,0.0379882
5,2001-05-31,-0.00663584
6,2001-06-29,-0.0252768
7,2001-07-31,0.0441707
8,2001-08-31,-0.0513603
9,2001-09-28,-0.0986025


In [311]:
describe(returns[:,:acwipln_returns])

Summary Stats:
Length:         280
Missing Count:  0
Mean:           0.004948
Std. Deviation: 0.037267
Minimum:        -0.111985
1st Quartile:   -0.015893
Median:         0.007448
3rd Quartile:   0.029379
Maximum:        0.105947
Type:           Float64


In [312]:
init_guess = [0.5, 0.5]
trans_guess = [0.8 0.2; 0.1 0.9]
dists_guess = [Normal(0.014,  0.0231), Normal(-0.0037, 0.0563)]
hmm_guess = HMM(init_guess, trans_guess, dists_guess); 

hmm_est, loglikelihood_evolution = baum_welch(hmm_guess, returns[:,:acwipln_returns]);
regime_summary(loglikelihood_evolution,hmm_est,12)

Row,loglikehood,mu1,mu2,p11,p21,sig1,sig2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,545.577,0.141836,-0.149567,0.959028,0.115111,0.0912319,0.180332


In [313]:
initialization(hmm_est)

2-element Vector{Float64}:
 2.176047869377171e-20
 1.0

In [314]:
init_guess = [0.4, 0.3,0.3]
trans_guess = [0.9 0.05 0.05; 0.05 0.9 0.05; 0.05 0.05 0.9]
dists_guess = [Normal(0.02,  0.0231), Normal(-0.0047, 0.0563),Normal(-0.01, 0.0363)]
hmm_guess = HMM(init_guess, trans_guess, dists_guess); 

hmm_est, loglikelihood_evolution = baum_welch(hmm_guess, returns[:,:acwipln_returns]);
last(loglikelihood_evolution)

547.0177325949245

In [315]:
transition_matrix(hmm_est)

3×3 Matrix{Float64}:
 0.894816  1.70974e-5  0.105167
 0.116467  0.883533    1.31627e-9
 0.33138   0.270612    0.398008

In [316]:
dist = obs_distributions(hmm_est)

3-element Vector{Normal{Float64}}:
 Normal{Float64}(μ=0.014372111229546377, σ=0.026733722881122315)
 Normal{Float64}(μ=-0.013085639896953175, σ=0.052501623626126794)
 Normal{Float64}(μ=-0.0028442204765647973, σ=0.01684757321985796)

In [317]:
println("regime 1")
println(mean(obs_distributions(hmm_est)[1])*12)
println(std(obs_distributions(hmm_est)[1])* 12^0.5)
println("regime 2")
println(mean(obs_distributions(hmm_est)[2])*12)
println(std(obs_distributions(hmm_est)[2])* 12^0.5)
println("regime 3")
println(mean(obs_distributions(hmm_est)[3])*12)
println(std(obs_distributions(hmm_est)[3])* 12^0.5)

regime 1
0.17246533475455653
0.09260833261114096
regime 2
-0.1570276787634381
0.1818709592006203
regime 3
-0.034130645718777566
0.05836170560206153


In [318]:
short = filter(row -> row.date >= Date(2003,01,01),returns)

Row,date,acwipln_returns
,Date,Float64
1,2003-01-31,-0.0317079
2,2003-02-28,0.00510341
3,2003-03-31,0.043069
4,2003-04-30,0.0118411
5,2003-05-30,0.0300992
6,2003-06-30,0.0670024
7,2003-07-31,0.0132941
8,2003-08-29,0.0495565
9,2003-09-30,0.000195124


In [319]:
init_guess = [0.5, 0.5]
trans_guess = [0.9 0.1; 0.1 0.9]
dists_guess = [Normal(0.014,  0.0231), Normal(-0.0037, 0.0563)]
hmm_guess = HMM(init_guess, trans_guess, dists_guess); 

hmm_est, loglikelihood_evolution = baum_welch(hmm_guess, short[:,:acwipln_returns]);
regime_summary(loglikelihood_evolution,hmm_est,12)

Row,loglikehood,mu1,mu2,p11,p21,sig1,sig2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,511.173,0.138062,-0.097021,0.960855,0.1387,0.0915307,0.181959


In [320]:
best_state_seq, best_joint_loglikelihood = viterbi(hmm_est, short[:,:acwipln_returns]);
short.states = best_state_seq

256-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [321]:
short[52:80,:]

Row,date,acwipln_returns,states
,Date,Float64,Int64
1,2007-04-30,0.00330561,1
2,2007-05-31,0.0520554,1
3,2007-06-29,-0.0209173,1
4,2007-07-31,-0.0220523,1
5,2007-08-31,0.011298,1
6,2007-09-28,-0.00506192,1
7,2007-10-31,-0.0166148,1
8,2007-11-30,-0.0658468,2
9,2007-12-31,-0.0153325,2


In [322]:
short.states = best_state_seq

256-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [323]:
acwi_long = DataFrame(XLSX.readdata("acwi_15.05.24.xlsx","price!A8:B288"), [:date,:acwi])
acwi_long.date = Date.(acwi_long.date,dateformat"u dd, yyyy") 
acwi_long.acwi = parse.(Float64,string.(acwi_long.acwi))
acwi_long

Row,date,acwi
,Date,Float64
1,1987-12-31,100.0
2,1988-01-29,102.33
3,1988-02-29,108.073
4,1988-03-31,111.222
5,1988-04-29,112.448
6,1988-05-31,110.041
7,1988-06-30,109.746
8,1988-07-29,111.593
9,1988-08-31,105.278


In [324]:
returns = log.(acwi_long.acwi[2:end]./acwi_long.acwi[1:end-1])
returns = DataFrame([data.date[2:end],returns],[:date,:acwi_returns,])

Row,date,acwi_returns
,Date,Float64
1,2001-01-31,0.0230327
2,2001-02-28,0.054604
3,2001-03-30,0.0287213
4,2001-04-30,0.0109627
5,2001-05-31,-0.0216379
6,2001-06-29,-0.00268442
7,2001-07-31,0.0166897
8,2001-08-31,-0.0582539
9,2001-09-28,0.0403971


In [325]:
mean_and_std(returns[:,:acwi_returns]) .* [12,12^0.5]

2-element Vector{Float64}:
 0.05452675921298197
 0.1574249398713421

In [340]:
describe(returns[:,:acwi_returns])

Summary Stats:
Length:         280
Missing Count:  0
Mean:           0.004544
Std. Deviation: 0.045445
Minimum:        -0.221961
1st Quartile:   -0.020665
Median:         0.010743
3rd Quartile:   0.033443
Maximum:        0.108725
Type:           Float64


In [327]:
init_guess = [0.5, 0.5]
trans_guess = [0.9 0.1; 0.1 0.9]
dists_guess = [Normal(0.014,  0.0231), Normal(-0.0037, 0.0563)]
hmm_guess = HMM(init_guess, trans_guess, dists_guess); 

hmm_est, loglikelihood_evolution = baum_welch(hmm_guess, returns[:,:acwi_returns]);
r1 = regime_summary(loglikelihood_evolution,hmm_est,12)

Row,loglikehood,mu1,mu2,p11,p21,sig1,sig2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,493.823,0.145617,-0.0668756,0.961666,0.0489704,0.0990306,0.205877


In [341]:
function guess_init(n_scenarios)
    regimes_probs = rand(1:100,n_scenarios)
    return regimes_probs/sum(regimes_probs)
end

function guess_tmatrix(n_scenarios)
    regimes_probs = rand(1:100,n_scenarios,n_scenarios)
    return regimes_probs ./ sum(regimes_probs,dims=2)
end

function guess_distribution()
    sigma = rand(Uniform(0,0.25))
    means = rand(Uniform(-0.25,0.25))
    return Normal(means,sigma)
end

guess_distribution (generic function with 1 method)

In [355]:
n_scenarios = 2
init_guess = guess_init(n_scenarios)
guess_matrix = guess_tmatrix(n_scenarios)
guess_dist = [guess_distribution() for s in 1:n_scenarios]
hmm_guess = HMM(init_guess, guess_matrix, guess_dist);
hmm_est, loglikelihood_evolution = baum_welch(hmm_guess, returns[:,:acwi_returns]);
r1 = regime_summary(loglikelihood_evolution,hmm_est,12)

Row,loglikehood,mu1,mu2,p11,p21,sig1,sig2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,493.823,0.145618,-0.0668837,0.961666,0.0489745,0.0990317,0.205879
